In [16]:
import pandas as pd
import numpy as np
import multiprocessing as mp
from pykrx import stock

In [13]:
# 날짜 설정 
start_date = '20220101'
end_date = '20241231'

In [17]:
# ETF 데이터 가져오기 함수
def fetch_etf_data(etf_ticker):
    try:
        data = stock.get_etf_ohlcv_by_date(start_date, end_date, etf_ticker)
        if not data.empty and "종가" in data.columns:
            return etf_ticker, data["종가"]
        return etf_ticker, None
    except Exception as e:
        print(f"Error fetching {etf_ticker}: {e}")
        return etf_ticker, None
    
# 포트폴리오 데이터 확인 함수
def check_portfolio_data(etf_ticker, date="20241210"):
    try:
        portfolio = stock.get_etf_portfolio_deposit_file(etf_ticker,date)
        if not portfolio.empty and "비중" in portfolio.columns and portfolio["비중"].sum() > 0:
            valid_stocks = portfolio[portfolio.index.str.match(r"^\d{6}$") & (portfolio["비중"] > 0)]
            if not valid_stocks.empty:
                return etf_ticker, True, valid_stocks
        return etf_ticker, False, None
    except Exception as e:
        print(f"Error checking portfolio for {etf_ticker}: {e}")
        return etf_ticker, False, None

if __name__=="__main__":
    # 모든 ETF 티커 가져오기
    etf_list = stock.get_etf_ticker_list()
    
    # 종가 데이터 가져오기
    with mp.Pool(processes=12) as pool:
        results = pool.map(fetch_etf_data, etf_list)
        
    # 유효한 데이터만 필터링
    etf_prices ={ticker: data for ticker, data in results if isinstance(data, pd.Series) and not data.empty}
    df_etf = pd.DataFrame(etf_prices)
    
    # 평균 종가 기준 최하위 10개 ETF 찾기
    etf_avg_prices = df_etf.mean()
    lowest_etfs = etf_avg_prices.nsmallest(10)
    print("최하위 10개 ETF (평균 종가 기준):")
    for ticker in lowest_etfs.index:
        name = stock.get_etf_ticker_name(ticker)
        print(f"{ticker}: {name} - 평균 종가: {lowest_etfs[ticker]:.2f}")
        
    # 포트폴리오 데이터 체크
    print("\n포트폴리오 데이터가 유효한 ETF 확인:")
    with mp.Pool(processes=12) as pool:
        portfolio_results = pool.starmap(check_portfolio_data, [(ticker, "20241010") for ticker in lowest_etfs.index])
    
    # 결과 분석
    valid_etfs = [(ticker, portfolio) for ticker, valid, portfolio in portfolio_results if valid]        
    if not valid_etfs:
        print("최하위 10개 ETF 중 유효한 비중 데이터를 제공하는 ETF가 없습니다.")
    else:
        print("유효한 비중 데이터를 제공하는 최하위 ETF:")
        for ticker, portfolio in valid_etfs:
            name = stock.get_etf_ticker_name(ticker)
            print(f"\n{ticker}: {name}")
            # 컬럼 확인 후 출력
            available_columns = [col for col in ["종목명", "비중"] if col in portfolio.columns]
            if not available_columns:
                print("출력할 유효한 컬럼이 없습니다.")
            else:
                print(portfolio[available_columns].head())

최하위 10개 ETF (평균 종가 기준):
204450: KODEX 차이나H레버리지(H) - 평균 종가: 2118.39
228790: TIGER 화장품 - 평균 종가: 2501.76
252420: RISE 200선물인버스2X - 평균 종가: 2597.76
252670: KODEX 200선물인버스2X - 평균 종가: 2604.34
253230: KIWOOM 200선물인버스2X - 평균 종가: 2606.46
252710: TIGER 200선물인버스2X - 평균 종가: 2743.28
139220: TIGER 200 건설 - 평균 종가: 2970.18
412560: TIGER BBIG레버리지 - 평균 종가: 3135.63
217770: TIGER 원유선물인버스(H) - 평균 종가: 3148.64
117700: KODEX 건설 - 평균 종가: 3182.97

포트폴리오 데이터가 유효한 ETF 확인:
유효한 비중 데이터를 제공하는 최하위 ETF:

228790: TIGER 화장품
           비중
티커           
214450  11.37
051900  10.77
257720   9.98
161890   9.97
278470   9.70

139220: TIGER 200 건설
               비중
티커               
000720  19.200001
028050  19.020000
028260  18.160000
052690  12.920000
006360  12.290000

117700: KODEX 건설
           비중
티커           
028050  18.51
000720  16.99
052690   9.42
006360   8.95
294870   6.34


In [19]:
import matplotlib.pyplot as plt

# 설정
etf_ticker = "139220" # 다른 부분
risk_free_rate = 0.02 # 무위험 수익률
date = "20241210" # 포트폴리오 데이터 기준 날짜

# 1. ETF 구성 종목과 비중 가져오기
portfolio = stock.get_etf_portfolio_deposit_file(etf_ticker, date)
valid_portfolio = portfolio[portfolio.index.str.match(r"^\d{6}$") & (portfolio["비중"] > 0 )].copy()
initial_tickers = valid_portfolio.index.tolist() # 초기 종목 코드
initial_weights = valid_portfolio["비중"].values / 100 # 초기 비중 (소수로 변환)

# 2. 구성 종목의 종가 데이터 가져오기
price_data = {}
for ticker in initial_tickers:
    try:
        df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
        if not df.empty and "종가" in df.columns:
            price_data[ticker] = df["종가"]
    except Exception as e:
        print(f"Error fetching data for {ticker}:{e}")

In [21]:
# 데이터프레임 결합
prices = pd.DataFrame(price_data)
# 결측값 제거 후 유효한 종목만 추출
prices = prices.dropna()
tickers = prices.columns.tolist() # 유효한 종목만 사용
print(f"유효한 종목 수: {len(tickers)}") # 디버깅용

유효한 종목 수: 8


In [22]:
# 초기 비중을 유효한 종목에 맞춰 조정
valid_indices = [initial_tickers.index(ticker) for ticker in tickers if ticker in initial_tickers]
initial_weights = initial_weights[valid_indices]
initial_weights /= initial_weights.sum()

# 3. 일별 수익률 계산
returns = prices.pct_change().dropna()
returns

,000720,028260,028050,006360,052690,375500,047040,300720
날짜,,,,,,,,
2022-01-04,0.008667,0.000000,0.013100,0.036205,-0.043084,0.024484,0.018739,0.054762
2022-01-05,0.003222,0.000000,-0.002155,0.028916,-0.068720,0.003994,0.000000,0.173815
2022-01-06,0.004283,-0.021277,0.053996,0.024590,-0.041985,0.035706,0.046823,-0.011538
2022-01-07,-0.001066,0.013043,0.008197,0.013714,0.026560,0.011492,-0.014377,-0.145914
2022-01-10,-0.020277,-0.012876,-0.010163,0.000000,0.019405,0.007579,0.009724,0.002278
...,...,...,...,...,...,...,...,...
2024-12-23,0.011628,0.015398,0.002943,0.021910,0.051852,0.037618,0.010606,-0.001218
2024-12-24,-0.003831,-0.009267,0.002934,-0.004948,-0.015845,-0.004532,-0.007496,0.000000
2024-12-26,-0.009615,0.000000,-0.027501,-0.020442,-0.025045,-0.016692,-0.022659,-0.003659


In [ ]:
# 4. 포트폴리오 성과 계산 함수
def portfolio_performance(weights, returns, risk_free_rate):
    portfolio_returns = returns.dot(weights)
    mean_return = portfolio_returns.mean() * 252 # 연율화 수익률
    portfolio_std = portfolio_returns.std() * np.sqrt(252) # 연율화 변동성
    sharpe_ratio = (mean_return - risk_free_rate) / portfolio_std
    return mean_return, portfolio_std, sharpe_ratio

In [25]:
# 5. Monte Carlo 시뮬레이션
num_portfolios = 1000000
results = np.zeros((3, num_portfolios))
weights_record = []

for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights /= weights.sum()
    weights_record.append(weights)
    mean_return, std, sharpe = portfolio_performance(weights, returns, risk_free_rate)
    results[0, i] = mean_return
    results[1, i] = std
    results[2, i] = sharpe    